In [1]:
import re
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import string

from nltk.corpus import stopwords
import nltk
import numba
from numba import jit

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [2]:
!unzip english_dataset.zip
!rm -rf english_dataset.zip

Archive:  english_dataset.zip
   creating: english_dataset/
  inflating: __MACOSX/._english_dataset  
  inflating: english_dataset/english_dataset.tsv  
  inflating: __MACOSX/english_dataset/._english_dataset.tsv  
  inflating: english_dataset/hasoc2019_en_test-2919.tsv  
  inflating: __MACOSX/english_dataset/._hasoc2019_en_test-2919.tsv  


In [3]:
import pandas as pd
data_en = pd.read_csv('english_dataset/english_dataset.tsv',delimiter='\t')
data_en.head(10)

,text_id,text,task_1,task_2,task_3
0,hasoc_en_1,#DhoniKeepsTheGlove | WATCH: Sports Minister K...,NOT,NONE,NONE
1,hasoc_en_2,@politico No. We should remember very clearly ...,HOF,HATE,TIN
2,hasoc_en_3,@cricketworldcup Guess who would be the winner...,NOT,NONE,NONE
3,hasoc_en_4,Corbyn is too politically intellectual for #Bo...,NOT,NONE,NONE
4,hasoc_en_5,All the best to #TeamIndia for another swimmin...,NOT,NONE,NONE
5,hasoc_en_6,@kellymiller513 @TheRealOJ32 I hope you rememb...,NOT,NONE,NONE
6,hasoc_en_7,@ICC Latest design of #WC2019 trophy. #CWC2019...,NOT,NONE,NONE
7,hasoc_en_8,#ADOS #trendingnow #blacklivesmatter #justice ...,HOF,PRFN,TIN
8,hasoc_en_9,Thanks for your support! Wow 600k. Graffiti ha...,NOT,NONE,NONE
9,hasoc_en_10,By wearing the #BalidaanBadge over his gloves ...,NOT,NONE,NONE


# Converting to Lowercase

In [4]:
data_lower = data_en
data_lower["text"].str.lower()

0       #dhonikeepstheglove | watch: sports minister k...
1       @politico no. we should remember very clearly ...
2       @cricketworldcup guess who would be the winner...
3       corbyn is too politically intellectual for #bo...
4       all the best to #teamindia for another swimmin...
                              ...                        
5847    @davidfrum @trueblueusa1 that's cute and all, ...
5848    a recession issa comin' #maga #magamyass #fuck...
5849    #doctorsfightback  will 'the mad n irrational ...
5850    #shiningindia #educatedindia or more like rund...
5851    could this be our new prime minister?     #ric...
Name: text, Length: 5852, dtype: object

# Converting to UpperCase

In [5]:
data_upper = data_en
data_upper["text"].str.upper()

0       #DHONIKEEPSTHEGLOVE | WATCH: SPORTS MINISTER K...
1       @POLITICO NO. WE SHOULD REMEMBER VERY CLEARLY ...
2       @CRICKETWORLDCUP GUESS WHO WOULD BE THE WINNER...
3       CORBYN IS TOO POLITICALLY INTELLECTUAL FOR #BO...
4       ALL THE BEST TO #TEAMINDIA FOR ANOTHER SWIMMIN...
                              ...                        
5847    @DAVIDFRUM @TRUEBLUEUSA1 THAT'S CUTE AND ALL, ...
5848    A RECESSION ISSA COMIN' #MAGA #MAGAMYASS #FUCK...
5849    #DOCTORSFIGHTBACK  WILL 'THE MAD N IRRATIONAL ...
5850    #SHININGINDIA #EDUCATEDINDIA OR MORE LIKE RUND...
5851    COULD THIS BE OUR NEW PRIME MINISTER?     #RIC...
Name: text, Length: 5852, dtype: object

# Removing Numbers

In [37]:
import re
data_n = data_en
result = re.sub(r'\d+', "", str(data_n["text"]))
print(result)

       #DhoniKeepsTheGlove | WATCH: Sports Minister K...
       @politico No. We should remember very clearly ...
       @cricketworldcup Guess who would be the winner...
       Corbyn is too politically intellectual for #Bo...
       All the best to #TeamIndia for another swimmin...
                              ...                        
    @davidfrum @trueblueusa That's cute and all, ...
    a recession issa comin' #maga #magamyass #fuck...
    #DoctorsFightBack  Will 'The Mad n Irrational ...
    #ShiningIndia #educatedindia or more like RUND...
    Could this be our new Prime Minister?     #Ric...
Name: text, Length: , dtype: object


# Removing Punctuations

In [39]:
import string
data_p = data_en
data_p = re.sub('[%s]' % re.escape(string.punctuation), ' ', str(data_p["text"]))
print(data_p)

0        DhoniKeepsTheGlove   WATCH  Sports Minister K   
1        politico No  We should remember very clearly    
2        cricketworldcup Guess who would be the winner   
3       Corbyn is too politically intellectual for  Bo   
4       All the best to  TeamIndia for another swimmin   
                                                         
5847     davidfrum  trueblueusa1 That s cute and all     
5848    a recession issa comin   maga  magamyass  fuck   
5849     DoctorsFightBack  Will  The Mad n Irrational    
5850     ShiningIndia  educatedindia or more like RUND   
5851    Could this be our new Prime Minister       Ric   
Name  text  Length  5852  dtype  object


# Removing Stop words

In [40]:
stop_words = stopwords.words("english")
data_sw = data_en
# Remove stop words
data_sw = ' '.join([word for word in str(data_sw["text"]).split(' ') if word not in stop_words])
print(data_sw)

0       #DhoniKeepsTheGlove | WATCH: Sports Minister K...
1       @politico No. We remember clearly ...
2       @cricketworldcup Guess would winner...
3       Corbyn politically intellectual #Bo...
4       All best #TeamIndia another swimmin...
                              ...                        
5847    @davidfrum @trueblueusa1 That's cute all, ...
5848    recession issa comin' #maga #magamyass #fuck...
5849    #DoctorsFightBack  Will 'The Mad n Irrational ...
5850    #ShiningIndia #educatedindia like RUND...
5851    Could new Prime Minister?     #Ric...
Name: text, Length: 5852, dtype: object


# Removing White Spaces

In [41]:
data_ws = data_en
data_ws = re.sub('\s{2,}', " ", str(data_ws["text"]))
print(data_ws)

0 #DhoniKeepsTheGlove | WATCH: Sports Minister K...
1 @politico No. We should remember very clearly ...
2 @cricketworldcup Guess who would be the winner...
3 Corbyn is too politically intellectual for #Bo...
4 All the best to #TeamIndia for another swimmin... ... 5847 @davidfrum @trueblueusa1 That's cute and all, ...
5848 a recession issa comin' #maga #magamyass #fuck...
5849 #DoctorsFightBack Will 'The Mad n Irrational ...
5850 #ShiningIndia #educatedindia or more like RUND...
5851 Could this be our new Prime Minister? #Ric...
Name: text, Length: 5852, dtype: object
